# <p style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;font-size:150%;text-align:center;border-radius:10px 10px;border-style:solid;border-color:#d90b1c;">Recommendation system for H and M Fashion</p>

<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">Terminologies</h1>

Có một số thuật ngữ nhất định cần được hiểu trước khi tiếp tục.

**Apache Spark:** Apache Spark là một khung điện toán cụm mục đích chung được phân phối mã nguồn mở. Nó cũng có thể được sử dụng với Hadoop.

**Collaborative filtering:** Collaborative filtering là phương pháp đưa ra dự đoán tự động (lọc) về sở thích của người dùng bằng cách thu thập thông tin sở thích hoặc sở thích từ nhiều người dùng. Xét ví dụ nếu một người A thích mục 1, 2, 3 và B thích 2,3,4 thì họ có cùng sở thích và A nên thích mục 4 và B nên thích mục 1.

**Alternating least square(ALS) matrix factorization:** Về cơ bản, ý tưởng là lấy một ma trận lớn (hoặc có khả năng rất lớn) và phân tích nó thành một số biểu diễn nhỏ hơn của ma trận ban đầu thông qua các bình phương nhỏ nhất xen kẽ. Chúng tôi kết thúc với hai hoặc nhiều ma trận chiều thấp hơn có tích bằng ma trận ban đầu.ALS được tích hợp sẵn trong Apache Spark.

**PySpark:** PySpark là sự hợp tác của Apache Spark và Python. PySpark là API Python cho Spark.

<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">1.Initialize spark session</h1>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 7.7 kB/s             ██████                         | 62.2 MB 5.9 MB/s eta 0:00:38
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 199 kB 49.2 MB/s            
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=a67156e44e01941da4d96ea946b57fd3f5ad055e0692ed6e763ff74fcea4cebd
  Stored in directory: /root/.cache/pip/wheels/07/fb/67/b9f2c0242d156eaa136b45ae4fd99d3e7c0ecc2acfd26f47b9
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.3
    Uninstalling py4j-0.10.9.3:
      Successfully uninstalled py4j-0.10.9.3



<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">2-Load libraries</h1>

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from pyspark.sql import SQLContext 
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf,col,when
from pyspark.sql.functions import to_timestamp,date_format
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

sc = SparkSession.builder.appName("Recommendations").config("spark.sql.files.maxPartitionBytes", 5000000).getOrCreate()
spark = SparkSession(sc)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/21 02:03:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable



<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">3-Load Dataset in Apache Spark</h1>

In [3]:
transaction = spark.read.option("header",True) \
              .csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
transaction.printSchema()

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



In [4]:
from pyspark.sql.functions import min, max
from pyspark.sql.functions import unix_timestamp, lit
min_date, max_date = transaction.select(min("t_dat"), max("t_dat")).first()
min_date, max_date

('2018-09-20', '2020-09-22')

<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">5-Select data for recommendation</h1>

Trong tập dữ liệu giao dịch này, chúng tôi có 31.788.324 hàng và 5 cột. Trước tiên hãy nắm bắt những bài báo được mua gần đây nhất là gì. Đối với khuyến nghị, tôi chỉ chọn ngày 22-09-2020 là ngày giao dịch cuối cùng.</h1>

In [5]:
hm =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
hm = hm.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
hm = hm.withColumn('year', year(col('date')))
hm = hm.withColumn('month', month(col('date')))
hm = hm.withColumn('day', date_format(col('date'), "d"))

hm = hm[hm['year'] == 2020]
hm = hm[hm['month'] == 9]
hm = hm[hm['day'] == 22]
transaction.unpersist()

# Prepare the dataset
hm = hm.groupby('customer_id', 'article_id').count()
hm.show(5)

+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|00f7bc5c0df4c615b...|0780418013|    1|
|02094817e46f3b692...|0791587001|    1|
|0333e5dda0257e9f4...|0839332002|    2|
|07c7a1172caf8fb97...|0573085043|    1|
|081373184e601470c...|0714790020|    1|
+--------------------+----------+-----+
only showing top 5 rows



In [6]:
print((hm.count(), len(hm.columns)))

(29486, 3)


In [7]:
# Count the total number of article in the dataset
numerator = hm.select("count").count()

# Count the number of distinct customerid and distinct articleid
num_users = hm.select("customer_id").distinct().count()
num_articles = hm.select("article_id").distinct().count()

# Set the denominator equal to the number of customer multiplied by the number of articles
denominator = num_users * num_articles

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("Sparsity: ", "%.2f" % sparsity + "%.")

Sparsity:  99.96%.


In [8]:
userId_count = hm.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|30b6056bacc5f5c9d...|   28|
|5e8fb4d457fdffc61...|   28|
|dc1b173e541f8d3c1...|   27|
|6335d496ef463bc40...|   25|
|1796e87fd2e88932b...|   25|
|f50287d9cf052d4b4...|   24|
|54e8ebd39543b5a4d...|   23|
|fd5ce8716faf00f6a...|   23|
|850ec77661a417d27...|   22|
|ad3663a848dccbdda...|   21|
|32f3a6a7ce63d302c...|   21|
|b606fe5786c00151a...|   21|
|298523b6637340717...|   21|
|b49647f84a99ced53...|   21|
|fc783381f1ea2174c...|   21|
|a08e284bb18add2d7...|   21|
|383e1b07e2c1fe169...|   21|
|3ca77aab50ae4532b...|   20|
|2a721767cd9864ed5...|   20|
|af5166e0f89b0d433...|   19|
+--------------------+-----+
only showing top 20 rows



In [9]:
articleId_count = hm.groupBy("article_id").count().orderBy('count', ascending=False)
articleId_count.show()

+----------+-----+
|article_id|count|
+----------+-----+
|0924243002|   91|
|0918522001|   88|
|0866731001|   78|
|0751471001|   75|
|0448509014|   73|
|0714790020|   72|
|0762846027|   68|
|0928206001|   67|
|0893432002|   66|
|0918292001|   65|
|0915529005|   64|
|0788575004|   63|
|0915529003|   63|
|0863583001|   60|
|0930380001|   59|
|0573085028|   59|
|0919273002|   58|
|0850917001|   57|
|0573085042|   56|
|0874110016|   53|
+----------+-----+
only showing top 20 rows



<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">5-Importing important modules</h1>

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS


<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">6-Converting String to index</h1>

Trước khi tạo một mô hình ALS, cần hiểu rõ rằng ALS chỉ chấp nhận các giá trị số nguyên làm tham số. Do đó, chúng ta cần chuyển đổi các cột customer_id và article_id thành dạng chỉ mục.

In [11]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(hm.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(hm).transform(hm)
transformed.show()

23/02/21 02:21:50 WARN DAGScheduler: Broadcasting large task binary with size 1210.9 KiB
+--------------------+----------+-----+-----------------+----------------+
|         customer_id|article_id|count|customer_id_index|article_id_index|
+--------------------+----------+-----+-----------------+----------------+
|00f7bc5c0df4c615b...|0780418013|    1|            783.0|          2237.0|
|02094817e46f3b692...|0791587001|    1|            785.0|            35.0|
|0333e5dda0257e9f4...|0839332002|    2|           4098.0|           732.0|
|07c7a1172caf8fb97...|0573085043|    1|           1702.0|            44.0|
|081373184e601470c...|0714790020|    1|           4146.0|             5.0|
|09bec2a61046ccbea...|0860336002|    1|           6792.0|          2368.0|
|0be4f1ecce204ee32...|0573085028|    1|            799.0|            14.0|
|0c4b30343292b5101...|0918522001|    1|           6825.0|             1.0|
|0e10e02358875468b...|0579541001|    1|           2689.0|            53.0|
|0fc371e67e


<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">7-Creating training and test data</h1>

In [12]:
(training,test)=transformed.randomSplit([0.8, 0.2])


<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">8-Creating ALS model and fitting data</h1>

Để xây dựng mô hình chỉ định rõ ràng các cột. Đặt giá trị không âm là 'True', vì chúng tôi đang tìm kiếm số lượng lớn hơn 0. Mô hình cũng cung cấp tùy chọn để chọn xếp hạng ẩn. Vì chúng tôi đang làm việc với tính rõ ràng, hãy đặt nó thành 'False' hoặc theo mặc định, nó sẽ rõ ràng.

Khi sử dụng các phân tách ngẫu nhiên đơn giản như trong Spark's CrossValidator hoặc TrainValidationSplit, thực tế rất phổ biến khi gặp phải người dùng và/hoặc các mục trong tập đánh giá không có trong tập huấn luyện. Theo mặc định, Spark chỉ định các dự đoán NaN trong ALSModel.transform khi yếu tố người dùng và/hoặc vật phẩm không có trong mô hình. Chúng tôi đặt chiến lược bắt đầu nguội thành 'drop' để đảm bảo chúng tôi không nhận được các chỉ số đánh giá NaN.

In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


#create ALS model
als=ALS(userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)

#tune model using ParamGridBuilder
param_grid = ParamGridBuilder()\
            .addGrid(als.rank, [15,20,25])\
            .addGrid(als.maxIter,[5,10,15])\
            .addGrid(als.regParam,[0.09,0.14,0.19])\
            .build()
#define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse",labelCol = 'count', predictionCol = 'prediction')

#Build cross validation using CrossValidator
cv = CrossValidator(estimator=als,estimatorParamMaps=param_grid, evaluator=evaluator,numFolds=3)


#Fit ALS model to training data
model = cv.fit(training)

23/02/21 02:23:31 WARN DAGScheduler: Broadcasting large task binary with size 1237.5 KiB


23/02/21 02:23:32 WARN DAGScheduler: Broadcasting large task binary with size 1238.9 KiB


23/02/21 02:23:40 WARN DAGScheduler: Broadcasting large task binary with size 1240.4 KiB


23/02/21 02:23:45 WARN DAGScheduler: Broadcasting large task binary with size 1241.7 KiB


23/02/21 02:23:45 WARN DAGScheduler: Broadcasting large task binary with size 1240.7 KiB


...

23/02/21 03:24:04 WARN DAGScheduler: Broadcasting large task binary with size 1271.8 KiB


23/02/21 03:24:05 WARN DAGScheduler: Broadcasting large task binary with size 1273.2 KiB


23/02/21 03:24:07 WARN DAGScheduler: Broadcasting large task binary with size 1274.6 KiB


23/02/21 03:24:07 WARN DAGScheduler: Broadcasting large task binary with size 1276.5 KiB


23/02/21 03:24:09 WARN DAGScheduler: Broadcasting large task binary with size 1275.1 KiB


In [14]:
# """"""als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
# model=als.fit(training)""""""


<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">9-Evaluate rmse</h1>

In [15]:
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = model.bestModel

#Generate predictions and evaluate using RMSE
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

23/02/21 03:24:10 WARN DAGScheduler: Broadcasting large task binary with size 1284.2 KiB
23/02/21 03:24:10 WARN DAGScheduler: Broadcasting large task binary with size 1282.8 KiB


23/02/21 03:25:50 WARN DAGScheduler: Broadcasting large task binary with size 1220.2 KiB


23/02/21 03:25:51 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


In [16]:
#print evaluation metrics and model parameters
print("RMSE =" + str(rmse))
print("**Best Model**")
print("Rank : {}".format(best_model.rank))
print("MaxIter: {}".format(best_model._java_obj.parent().getMaxIter()))
print("RegParam: {}".format(best_model._java_obj.parent().getRegParam()))

RMSE =0.43970652201919097
**Best Model**
Rank : 20
MaxIter: 15
RegParam: 0.09


In [17]:
predictions.show()

23/02/21 03:25:52 WARN DAGScheduler: Broadcasting large task binary with size 1284.2 KiB
23/02/21 03:25:52 WARN DAGScheduler: Broadcasting large task binary with size 1282.8 KiB


23/02/21 03:27:31 WARN DAGScheduler: Broadcasting large task binary with size 1221.3 KiB


+--------------------+----------+-----+-----------------+----------------+----------+
|         customer_id|article_id|count|customer_id_index|article_id_index|prediction|
+--------------------+----------+-----+-----------------+----------------+----------+
|290eb29441f631669...|0865935001|    1|           2866.0|           743.0| 0.8191221|
|47b77e73dd60193bc...|0826492001|    1|           1959.0|          1181.0| 0.8571001|
|b4db5e5259234574e...|0936726001|    1|           2366.0|          4079.0| 0.8764172|
|d97e200fbf2e50ba7...|0456163087|    1|           1088.0|           204.0| 1.0435058|
|d97e200fbf2e50ba7...|0909588006|    1|           1088.0|           514.0| 0.9705297|
|db1f0328dcdc4e123...|0801673001|    1|           1580.0|           333.0|0.86747843|
|e12e0cadbb3a8bbc7...|0920159001|    1|           6357.0|          2649.0|0.40935484|
|f5689b0053eb6f0bd...|0863583001|    1|           1645.0|            13.0| 1.2966778|
|ff94f31e864d9b655...|0884319003|    1|           6658

In [18]:
predictions.count()

23/02/21 03:27:32 WARN DAGScheduler: Broadcasting large task binary with size 1284.2 KiB
23/02/21 03:27:32 WARN DAGScheduler: Broadcasting large task binary with size 1282.8 KiB


23/02/21 03:29:11 WARN DAGScheduler: Broadcasting large task binary with size 1220.8 KiB


4077

<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">10-Providing Recommendations by Article id</h1>

In [19]:
user_recs=best_model.recommendForAllItems(10).show(10)

23/02/21 03:29:13 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


23/02/21 03:29:25 WARN DAGScheduler: Broadcasting large task binary with size 1307.6 KiB
+----------------+--------------------+
|article_id_index|     recommendations|
+----------------+--------------------+
|               1|[{4907, 5.1636834...|
|               3|[{9001, 5.862812}...|
|               5|[{4907, 5.9506326...|
|               6|[{9001, 5.1357713...|
|               9|[{9001, 5.409152}...|
|              12|[{9001, 5.4819713...|
|              13|[{9001, 5.5410566...|
|              15|[{4907, 5.258558}...|
|              16|[{9001, 5.557741}...|
|              17|[{4907, 6.4078674...|
+----------------+--------------------+
only showing top 10 rows




<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">11-Providing Recommendations by Customer id</h1>

In [20]:
df_recom = best_model.recommendForAllUsers(12)
df_recom.show(10)

23/02/21 03:29:25 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


23/02/21 03:29:36 WARN DAGScheduler: Broadcasting large task binary with size 1307.6 KiB
+-----------------+--------------------+
|customer_id_index|     recommendations|
+-----------------+--------------------+
|                1|[{6383, 2.3995473...|
|                3|[{1661, 2.5343025...|
|                5|[{6383, 3.2585394...|
|                6|[{6383, 6.869421}...|
|                9|[{6383, 2.120083}...|
|               12|[{6383, 2.3262966...|
|               13|[{6383, 2.6276238...|
|               15|[{6383, 2.3234072...|
|               16|[{6383, 2.4736617...|
|               17|[{6383, 2.4062884...|
+-----------------+--------------------+
only showing top 10 rows



In [31]:
test = best_model.recommendForAllUsers(10)
test.show()

23/02/21 03:49:02 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


23/02/21 03:49:15 WARN DAGScheduler: Broadcasting large task binary with size 1307.6 KiB
+-----------------+--------------------+
|customer_id_index|     recommendations|
+-----------------+--------------------+
|                1|[{6383, 2.3995473...|
|                3|[{1661, 2.5343025...|
|                5|[{6383, 3.2585394...|
|                6|[{6383, 6.869421}...|
|                9|[{6383, 2.120083}...|
|               12|[{6383, 2.3262966...|
|               13|[{6383, 2.6276238...|
|               15|[{6383, 2.3234072...|
|               16|[{6383, 2.4736617...|
|               17|[{6383, 2.4062884...|
|               19|[{4249, 2.7245257...|
|               20|[{6383, 2.1003773...|
|               22|[{6383, 3.4867506...|
|               26|[{6383, 2.283442}...|
|               27|[{6383, 2.317651}...|
|               28|[{6383, 2.2771437...|
|               31|[{1661, 1.9318609...|
|               34|[{6383, 2.1202626...|
|               35|[{3870, 2.703423}...|
|        

In [22]:
test.count()

23/02/21 03:29:46 WARN DAGScheduler: Broadcasting large task binary with size 1328.6 KiB


23/02/21 03:29:53 WARN DAGScheduler: Broadcasting large task binary with size 1308.4 KiB


9612

In [23]:
df_recom = df_recom.select("customer_id_index","recommendations.article_id_index")
df_recom.show(10)
df_recom = df_recom.toPandas()

23/02/21 03:29:53 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


23/02/21 03:30:03 WARN DAGScheduler: Broadcasting large task binary with size 1307.9 KiB
+-----------------+--------------------+
|customer_id_index|    article_id_index|
+-----------------+--------------------+
|                1|[6383, 1661, 4405...|
|                3|[1661, 5111, 4405...|
|                5|[6383, 5111, 4405...|
|                6|[6383, 1910, 1661...|
|                9|[6383, 1661, 4405...|
|               12|[6383, 1661, 4405...|
|               13|[6383, 1661, 4405...|
|               15|[6383, 1661, 4405...|
|               16|[6383, 1661, 4405...|
|               17|[6383, 1661, 4405...|
+-----------------+--------------------+
only showing top 10 rows



23/02/21 03:30:03 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


23/02/21 03:30:13 WARN DAGScheduler: Broadcasting large task binary with size 1307.8 KiB


In [24]:
df_recom.sort_values('customer_id_index')

,customer_id_index,article_id_index
4762,0,"[6383, 1661, 4405, 5111, 7073, 4249, 3674, 387..."
0,1,"[6383, 1661, 4405, 5111, 1910, 4249, 3674, 707..."
4763,2,"[6383, 4249, 4405, 5111, 1661, 2874, 6046, 367..."
1,3,"[1661, 5111, 4405, 6383, 3870, 7073, 6046, 367..."
4764,4,"[1661, 5111, 4405, 6383, 4249, 1891, 7073, 367..."
...,...,...
9610,10522,"[6383, 1661, 4405, 5111, 3870, 7073, 6046, 367..."
4759,10524,"[6383, 1661, 5111, 4249, 4405, 3674, 7864, 707..."
9611,10525,"[1661, 1221, 6383, 4212, 4405, 1910, 3229, 598..."
4760,10526,"[6848, 6383, 3663, 354, 190, 3787, 6975, 1332,..."


In [32]:
test = test.select("customer_id_index","recommendations.article_id_index")
test.show(10)
# df_recom = df_recom.toPandas()

23/02/21 03:54:47 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


23/02/21 03:54:56 WARN DAGScheduler: Broadcasting large task binary with size 1307.9 KiB
+-----------------+--------------------+
|customer_id_index|    article_id_index|
+-----------------+--------------------+
|                1|[6383, 1661, 4405...|
|                3|[1661, 5111, 4405...|
|                5|[6383, 5111, 4405...|
|                6|[6383, 1910, 1661...|
|                9|[6383, 1661, 4405...|
|               12|[6383, 1661, 4405...|
|               13|[6383, 1661, 4405...|
|               15|[6383, 1661, 4405...|
|               16|[6383, 1661, 4405...|
|               17|[6383, 1661, 4405...|
+-----------------+--------------------+
only showing top 10 rows



<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">12-Converting back to string form</h1>

As seen in above image the results are in integer form we need to convert it back to its original name.The code is little bit longer given so many conversions.

In [26]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import pandas as pd
md=transformed.select(transformed['article_id'],transformed['article_id_index'],transformed['customer_id'],transformed['customer_id_index'])
md=md.toPandas()
md

23/02/21 03:32:05 WARN DAGScheduler: Broadcasting large task binary with size 1207.8 KiB


,article_id,article_id_index,customer_id,customer_id_index
0,0780418013,2237.0,00f7bc5c0df4c615b2502a2c2e9ef9eff988c81dec2e5e...,783.0
1,0791587001,35.0,02094817e46f3b692149b06cf9577e42848c2294e78598...,785.0
2,0839332002,732.0,0333e5dda0257e9f498be52f1e569bfae576caed0cbdcd...,4098.0
3,0573085043,44.0,07c7a1172caf8fb9784b28e51b25b985ab6a1ec7ce923e...,1702.0
4,0714790020,5.0,081373184e601470cc9911f33d3eeebc6f33ed79222573...,4146.0
...,...,...,...,...
29481,0817150004,851.0,f8156f726aeaf44e90c1988837e13c9c9974ee19009e5b...,2594.0
29482,0893432005,96.0,f825202d015506981dc42c53afb7b56a36e05c85b67886...,1657.0
29483,0799754001,5499.0,faed38aaccd80db66f5a1581fe99af84e79fe398c91899...,10465.0
29484,0928206001,7.0,fcffcb9777aab7a53e3b382a840958d800e6d53bdd8a20...,4052.0


In [27]:
test = test.toPandas()

23/02/21 03:32:07 WARN DAGScheduler: Broadcasting large task binary with size 1329.9 KiB


23/02/21 03:32:18 WARN DAGScheduler: Broadcasting large task binary with size 1307.8 KiB


In [28]:
dict1 =dict(zip(md['article_id_index'],md['article_id']))
dict2=dict(zip(md['customer_id_index'],md['customer_id']))
test['prediction'] = test['article_id_index'].map(lambda x: [dict1[y] for y in x if y in dict1])
test['customer_id']=test['customer_id_index'].map(dict2)
test

,customer_id_index,article_id_index,prediction,customer_id
0,1,"[6383, 1661, 4405, 5111, 1910, 4249, 3674, 707...","[0857347002, 0297078008, 0571048002, 075797100...",5e8fb4d457fdffc61e235328ba7e43a4139c94c5f9d52a...
1,3,"[1661, 5111, 4405, 6383, 3870, 7073, 6046, 367...","[0297078008, 0757971006, 0571048002, 085734700...",1796e87fd2e88932b50966a07cc18b490cd5e1474dbee3...
2,5,"[6383, 5111, 4405, 4249, 3870, 7864, 7073, 166...","[0857347002, 0757971006, 0571048002, 050286900...",f50287d9cf052d4b423fc3d4d7a0c306de8c752583544b...
3,6,"[6383, 1910, 1661, 4894, 2696, 1064, 4405, 206...","[0857347002, 0880479001, 0297078008, 072199000...",54e8ebd39543b5a4d69c3e7d79977558d2a606e6540ba0...
4,9,"[6383, 1661, 4405, 5111, 7073, 3870, 3674, 191...","[0857347002, 0297078008, 0571048002, 075797100...",298523b6637340717e19df4e2a46a7ce7d80434c985a84...
...,...,...,...,...
9607,10515,"[6383, 1661, 1910, 2163, 4405, 308, 3013, 99, ...","[0857347002, 0297078008, 0880479001, 069959800...",fefb56faca51b2e9de0082a3da3379e1fd41709509f6a4...
9608,10516,"[6383, 1910, 1661, 4405, 5111, 3870, 7073, 367...","[0857347002, 0880479001, 0297078008, 057104800...",ff09354db173e36e7148bd2da4da7890eaa95b00556014...
9609,10519,"[6383, 1661, 4405, 5111, 7073, 3674, 4249, 604...","[0857347002, 0297078008, 0571048002, 075797100...",ff240ee1590922141103063f7b4212c3832f0f5b0e0eb2...
9610,10522,"[6383, 1661, 4405, 5111, 3870, 7073, 6046, 367...","[0857347002, 0297078008, 0571048002, 075797100...",ff6f55a51af284b71dcd264396b299e548f968c1769e71...


In [29]:
recom_final = test.drop(['customer_id_index','article_id_index'], axis = 1)
finalpre=recom_final[['customer_id','prediction']]     
finalpre

,customer_id,prediction
0,5e8fb4d457fdffc61e235328ba7e43a4139c94c5f9d52a...,"[0857347002, 0297078008, 0571048002, 075797100..."
1,1796e87fd2e88932b50966a07cc18b490cd5e1474dbee3...,"[0297078008, 0757971006, 0571048002, 085734700..."
2,f50287d9cf052d4b423fc3d4d7a0c306de8c752583544b...,"[0857347002, 0757971006, 0571048002, 050286900..."
3,54e8ebd39543b5a4d69c3e7d79977558d2a606e6540ba0...,"[0857347002, 0880479001, 0297078008, 072199000..."
4,298523b6637340717e19df4e2a46a7ce7d80434c985a84...,"[0857347002, 0297078008, 0571048002, 075797100..."
...,...,...
9607,fefb56faca51b2e9de0082a3da3379e1fd41709509f6a4...,"[0857347002, 0297078008, 0880479001, 069959800..."
9608,ff09354db173e36e7148bd2da4da7890eaa95b00556014...,"[0857347002, 0880479001, 0297078008, 057104800..."
9609,ff240ee1590922141103063f7b4212c3832f0f5b0e0eb2...,"[0857347002, 0297078008, 0571048002, 075797100..."
9610,ff6f55a51af284b71dcd264396b299e548f968c1769e71...,"[0857347002, 0297078008, 0571048002, 075797100..."


<h1 style="background-color:#f7e9ec;font-family:newtimeroman;color:#d90b1c;">13-Export the prediction</h1>

In [30]:
# my_pred = finalpre.toPandas()
finalpre.to_csv('submission.csv', index=False)